# QLD Conservation Status and Sensitive Species Lists

This notebook downloads the Qld lists from the [Qld Government Open Data Portal](https://data.qld.gov.au) and formats them in Darwin Core for ingestion into the ALA Lists tool.
It will save original lists to the `source-data/QLD` directory, process the lists and save them to `current-lists`.


## Lists in the ALA Species List tool
* Conservation list: __[dr652](https://lists.ala.org.au/speciesListItem/list/dr652)__ ([dr652 in test](https://lists-test.ala.org.au/speciesListItem/list/dr652))
* Collection: __[dr652](https://collections.ala.org.au/public/show/dr652)__

## Source Data
Queensland Nature Conservation Act 1992

* __[Conservation](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species.csv)__
* __[Species Codes](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv)__

**Metadata Description**

**Conservation:** The list of taxa from the Department of Environment and Science’s WildNet database with their classification codes under the Nature Conservation Act 1992: Extinct (EX), Extinct in the wild (PE), Critically Endangered (CR), Endangered (E), Vulnerable (V), Near threatened (NT, Least concern (C), Special least concern (SL) and International (I). It is published weekly as the Conservation status of Queensland wildlife dataset in the Queensland Government Data portal. Learn more about the classifications used by the Department of Environment and Science from Listing and changing the conservation status of Queensland species.


**Metadata URL**
* Qld Species (Open Data Portal) https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife
* Queensland Confidential Species (Open Data Portal) https://www.data.qld.gov.au/dataset/queensland-confidential-species
* Qld Species codes https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife/resource/6344ea93-cadf-4e0c-9ff4-12dfb18d5f14


# Setup
* Import libraries
* Set Project directory
* Set URLs

In [10]:
import datetime

import pandas as pd
import requests
import io
from ftfy import fix_encoding
import urllib.request, json
import certifi
import ssl
import os
import sys

projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf

sourceDataDir = projectDir + "source-data/QLD/"
statusDir = projectDir + "source-data/status-codes/"
processedDataDir = projectDir + "current-lists/"
state = 'QLD'
monthStr = datetime.datetime.now().strftime('%Y%m')
codesfile = statusDir + state + "-codes.csv"
codesurl =  "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv"
listurl = "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species.csv"


## Download the raw files from data.qld.gov.au
... save locally

In [11]:
# %%script echo skipping # comment this line to download dataset from API

# Status codes
response = requests.get(codesurl)
rtext = fix_encoding(response.text)
speciescodes = pd.read_csv(io.StringIO(rtext))
speciescodes.to_csv(sourceDataDir + "species-status-codes.csv", index=False)

# Conservation List
response = requests.get(listurl)
rtext = fix_encoding(response.text)
conservationlist = pd.read_csv(io.StringIO(rtext))
conservationlist.to_csv(sourceDataDir + "species.csv", index=False)


## Standardise Status Codes
Some minimal changes to some Qld Nature Conservation Act codes so that they are consistent with other states

In [ ]:
# speciescodes = pd.read_csv(sourceDataDir + "species-status-codes.csv")

## Conservation List
* Read in the Conservation list
* Join to the codes to expand the code descriptions.
* Change the field names to `sourceStatus` and `status` as required by the ALA's conservation list processing.
* Remove **Least concern**, **Special least concern** and **no status**
* Expand the endemicity and epbc status codes
* Map sourceStatus to status

## Option to read from local files for subsequent runs

In [ ]:
# conservationlist = pd.read_csv(sourceDataDir + "species.csv")


In [3]:
conservationlist

,Taxon_Id,Kingdom,Class,Family,Scientific_name,Common_name,Taxon_author,NCA_status,EPBC_status,Significant,Confidential,Endemicity
0,706,animals,amphibians,Limnodynastidae,Adelotus brevis,tusked frog,"(Günther, 1863)",V,NaN,Y,N,QA
1,690,animals,amphibians,Limnodynastidae,Lechriodus fletcheri,black soled frog,"(Boulenger, 1890)",C,NaN,N,N,QA
2,677,animals,amphibians,Limnodynastidae,Limnodynastes convexiusculus,marbled frog,"(MacLeay, 1877)",C,NaN,N,N,QAI
3,678,animals,amphibians,Limnodynastidae,Limnodynastes dumerilii,grey bellied pobblebonk,"Peters, 1863",C,NaN,N,N,QA
4,679,animals,amphibians,Limnodynastidae,Limnodynastes fletcheri,barking frog,"Boulenger, 1888",C,NaN,N,N,QA
...,...,...,...,...,...,...,...,...,...,...,...,...
21889,25858,protozoans,slime molds,Physaraceae,Fuligo septica,NaN,(L.) F.H.Wigg.,C,NaN,N,N,Q
21890,34854,protozoans,slime molds,Stemonitidaceae,Comatricha pulchella,NaN,(Bab.) Rostaf.,C,NaN,N,N,Q
21891,27732,protozoans,slime molds,Stemonitidaceae,Stemonitis splendens,NaN,Rostaf.,C,NaN,N,N,Q
21892,41204,protozoans,slime molds,Trichiidae,Hemitrichia serpula,NaN,(Scop.) Rostaf.,C,NaN,N,N,Q


In [4]:
conservationlist = conservationlist.rename(columns=
{   'Scientific_name':'scientificName',
    'Common_name': 'vernacularName',
    'Taxon_author':'scientificNameAuthorship',
    'Family': 'family',
    'NCA_status':'sourceStatus'
})

In [5]:
conservationlist.columns

Index(['Taxon_Id', 'Kingdom', 'Class', 'family', 'scientificName',
       'vernacularName', 'scientificNameAuthorship', 'sourceStatus',
       'EPBC_status', 'Significant', 'Confidential', 'Endemicity'],
      dtype='object')

## Map status codes
* Map to codes in provided state based codes file
* Remove records with statuses of:
  * Special least concern
  * Least concert
  * null/na

In [6]:
conservationlist = lf.map_status(state, codesfile, conservationlist)
conservationlist

,Taxon_Id,Kingdom,Class,family,scientificName,vernacularName,scientificNameAuthorship,sourceStatus,EPBC_status,Significant,Confidential,Endemicity,status
0,706,animals,amphibians,Limnodynastidae,Adelotus brevis,tusked frog,"(Günther, 1863)",V,NaN,Y,N,QA,Vulnerable
1,690,animals,amphibians,Limnodynastidae,Lechriodus fletcheri,black soled frog,"(Boulenger, 1890)",C,NaN,N,N,QA,Least concern
2,677,animals,amphibians,Limnodynastidae,Limnodynastes convexiusculus,marbled frog,"(MacLeay, 1877)",C,NaN,N,N,QAI,Least concern
3,678,animals,amphibians,Limnodynastidae,Limnodynastes dumerilii,grey bellied pobblebonk,"Peters, 1863",C,NaN,N,N,QA,Least concern
4,679,animals,amphibians,Limnodynastidae,Limnodynastes fletcheri,barking frog,"Boulenger, 1888",C,NaN,N,N,QA,Least concern
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21889,25858,protozoans,slime molds,Physaraceae,Fuligo septica,NaN,(L.) F.H.Wigg.,C,NaN,N,N,Q,Least concern
21890,34854,protozoans,slime molds,Stemonitidaceae,Comatricha pulchella,NaN,(Bab.) Rostaf.,C,NaN,N,N,Q,Least concern
21891,27732,protozoans,slime molds,Stemonitidaceae,Stemonitis splendens,NaN,Rostaf.,C,NaN,N,N,Q,Least concern
21892,41204,protozoans,slime molds,Trichiidae,Hemitrichia serpula,NaN,(Scop.) Rostaf.,C,NaN,N,N,Q,Least concern


In [7]:
conservationlist = conservationlist[((conservationlist['status'].notna()))]
conservationlist = conservationlist[~conservationlist['status'].str.contains('Special least concern', case=False)]
conservationlist = conservationlist[~conservationlist['status'].str.contains('Least concern', case=False)]
conservationlist

,Taxon_Id,Kingdom,Class,family,scientificName,vernacularName,scientificNameAuthorship,sourceStatus,EPBC_status,Significant,Confidential,Endemicity,status
0,706,animals,amphibians,Limnodynastidae,Adelotus brevis,tusked frog,"(Günther, 1863)",V,NaN,Y,N,QA,Vulnerable
15,687,animals,amphibians,Limnodynastidae,Philoria kundagungan,red-and-yellow mountainfrog,"(Ingram & Corben, 1975)",E,E,Y,Y,QA,Endangered
25,686,animals,amphibians,Myobatrachidae,Crinia tinnula,wallum froglet,"Straughan & Main, 1966",V,NaN,Y,N,QA,Vulnerable
29,675,animals,amphibians,Myobatrachidae,Mixophyes fleayi,Fleay's barred frog,"Corben & Ingram, 1987",E,E,Y,Y,QA,Endangered
30,676,animals,amphibians,Myobatrachidae,Mixophyes iteratus,giant barred frog,"Straughan, 1968",V,V,Y,Y,QA,Vulnerable
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21799,27445,plants,land plants,Zamiaceae,Macrozamia serpentina,NaN,D.L.Jones & P.I.Forst.,E,NaN,Y,Y,Q,Endangered
21800,6482,plants,land plants,Zamiaceae,Macrozamia viridis,NaN,D.L.Jones & P.I.Forst.,E,NaN,Y,Y,QA,Endangered
21805,8948,plants,land plants,Zingiberaceae,Alpinia hylandii,NaN,R.M.Sm.,NT,NaN,Y,N,Q,Near threatened
21808,8949,plants,land plants,Zingiberaceae,Amomum queenslandicum,NaN,R.M.Sm.,V,NaN,Y,N,Q,Vulnerable


**Tidy up**
* rename fields to Darwin Core
* replace kingdom/class values with scientific names

In [8]:
conservationlist = conservationlist.loc[:, ['scientificName', 'vernacularName', 'family', 'status', 'sourceStatus']]

KeyError: "['genus'] not in index"

Write dataframe to CSV - UTF-8 encoding

In [ ]:
conservationlist.to_csv(processedDataDir + "conservation-lists/QLD-conservation-20230601.csv",encoding="UTF-8",index=False)
print("Finished writing QLD conservation list")

In [ ]:
### Change logs

In [ ]:
ltype = "C"
changeDir = "Monitoring/Change-logs/"
# Qld Conservation
print('QLD conservation')
filename = "QLD-conservation.csv"
testdr = "dr652"
proddr = "dr652"
changelist = lf.get_changelist(testdr, proddr, ltype)
changelist.to_csv(projectDir + changeDir + monthStr + "-" + filename, encoding="UTF-8", index=False)
print('Finished QLD conservation')

In [ ]:
# Download historical lists from Production
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + proddr + "?max=10000&includeKVP=true"
prodList = lf.download_ala_specieslist(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/conservation/" + filename, encoding="UTF-8", index=False)
print('Finished downloading conservation historical list')